# Aqua / Terra MODIS NBAR Time Series via Sentinel-Hub Process API

### _Import functional dependencies and add repository source directory to system path_

In [1]:
import os
import yaml
from munch import munchify

In [2]:
# define repo name and get root working directory
repo = 'sentinelhub-py'
root_path = os.getcwd()[ 0 : os.getcwd().find( repo ) + len ( repo )]
root_path

'C:\\Users\\Chris.Williams\\Documents\\GitHub\\sentinelhub-py'

In [3]:
# add repo source + notebooks directory to system path
import sys
sys.path.insert(0, os.path.join( root_path, 'src' ) )
sys.path.insert(0, os.path.join( root_path, 'notebooks' ) )

In [4]:
# get path to configuration files
cfg_path = os.path.join( root_path, 'cfg' )
cfg_path = os.path.join( cfg_path, 'modis' )
cfg_path

'C:\\Users\\Chris.Williams\\Documents\\GitHub\\sentinelhub-py\\cfg\\modis'

### _Load configuration file from repository cfg sub-directory_ 

In [5]:
# get pathname to configuration file
cfg_file = os.path.join( cfg_path, 'modis-timeseries-nbar.yml' )

In [6]:
# print contents of configuration file
with open( cfg_file ) as f:
    print ( f.read() )

request:
    collections: MODIS
    evalscript: |
                //VERSION=3
                function setup() {
                    return {
                        input: [{
                            bands: ["B01", "B02", "B03", "B04", "B05", "B06", "B07" ],
                            units: ["reflectance", "reflectance", "reflectance", "reflectance", "reflectance", "reflectance", "reflectance" ] 
                        }],
                        output: {
                            bands: 7,
                            sampleType: "FLOAT32"
                        }
                    };
                }
                function evaluatePixel(sample) {
                    return [sample.B01, sample.B02, sample.B03, sample.B04, sample.B05, sample.B06, sample.B07 ];
                }
responses:
    default: TIFF
aoi:
    pathname: <path-to-aoi-file>
    name: name
    distance: 500



In [7]:
# load cfg file using yaml parser
with open( cfg_file, 'r' ) as f:
    config = munchify( yaml.safe_load( f ) )

### _Define area of interest centred on the Caspian Sea_

In [8]:
# define min and max latlons
coords = 44.666016,36.333218,59.343748,47.314992
resolution = 500

In [9]:
# create instance of shclient class
from client import ShClient
client = ShClient( config )

In [10]:
# get utm bbox
bbox = client.getBoundingBox( coords )
print ( bbox, bbox.crs.value )

-68802.60549586022,4039580.0442555356,1130416.3144492358,5274026.951890117 32639


In [11]:
# get aoi image dimensions
client.getBoxDimensions( bbox, resolution )

(2398, 2469)

### _Use Catalog API to locate MODIS scene acquired on August 2020_

In [12]:
# define aggregation timeframe
from datetime import datetime
timeframe = { 'start' : datetime.strptime('2020-08-10', '%Y-%m-%d'), 'end' : datetime.strptime('2020-08-11', '%Y-%m-%d') }

In [13]:
# get scenes from catalogue
client.getTimeStamps( bbox, timeframe )

AttributeError: collection

### _Download MODIS NBAR channel data via Process API client_

In [ ]:
# get mosaic between timeframe at specified pixel resolution
response = client.getTimeSeries( bbox, timeframe, resolution )
response._df

### _Visualise NBAR channel imagery with high reflectances (cloud) masked as no data_

In [ ]:
import numpy as np

# get top row data
response._df[ 'mask-default'] = response._df[ 'default' ].apply( lambda x: np.ma.masked_where( x > 0.8, x ) )
data = response._df.iloc[ 0 ][ 'mask-default' ]

# 7 band output image !
height, width, channels = data.shape
height, width, channels

In [ ]:
import matplotlib.pyplot as plt

# compute figure size
dpi = 80; border = 1
figsize = ( width + 2 * border) / float(dpi), ( ( height + 2 * border ) * channels ) / float(dpi)

# create figure and grid
fig, axes = plt.subplots(   figsize=figsize, 
                            nrows=channels, 
                            ncols=1, 
                            constrained_layout=True )

# iterate through channels
for idx in range( 0, channels ):

    # plot channel image
    axes[ idx ].set_title( f'MODIS Channel {idx+1}' )
    axes[ idx ].imshow( data[ :, :, idx ], 
                            interpolation='nearest', 
                            aspect='auto', 
                            cmap='gray' )    